In [10]:
!python3 -m pip install pip --upgrade
!python3 -m pip install opencv-python

In [6]:
import os
import glob
import json
from scipy.io import loadmat
from collections import defaultdict

all_obj = 0
data = defaultdict(dict)
for dname in sorted(glob.glob('data/annotations/set*')):
    set_name = os.path.basename(dname)
    data[set_name] = defaultdict(dict)
    for anno_fn in sorted(glob.glob('{}/*.vbb'.format(dname))):
        vbb = loadmat(anno_fn)
        nFrame = int(vbb['A'][0][0][0][0][0])
        objLists = vbb['A'][0][0][1][0]
        maxObj = int(vbb['A'][0][0][2][0][0])
        objInit = vbb['A'][0][0][3][0]
        objLbl = [str(v[0]) for v in vbb['A'][0][0][4][0]]
        objStr = vbb['A'][0][0][5][0]
        objEnd = vbb['A'][0][0][6][0]
        objHide = vbb['A'][0][0][7][0]
        altered = int(vbb['A'][0][0][8][0][0])
        log = vbb['A'][0][0][9][0]
        logLen = int(vbb['A'][0][0][10][0][0])

        video_name = os.path.splitext(os.path.basename(anno_fn))[0]
        data[set_name][video_name]['nFrame'] = nFrame
        data[set_name][video_name]['maxObj'] = maxObj
        data[set_name][video_name]['log'] = log.tolist()
        data[set_name][video_name]['logLen'] = logLen
        data[set_name][video_name]['altered'] = altered
        data[set_name][video_name]['frames'] = defaultdict(list)

        n_obj = 0

        for frame_id, obj in enumerate(objLists):
            # print("frame_id: {}".format(frame_id))
            if len(obj) > 0:
                # print(len(obj[0]))
                ### different between data-USA and data-INRIA annotations.
                # data-USA: id, pos, occl, lock, posv
                # data-INRIA: id, pos, posv, occl, lock
                for id, pos, posv, occl, lock in obj[0]:
                    #print(id)
                    #print(pos)
                    #print(posv)
                    #print(occl)
                    #print(lock)
                    keys = obj.dtype.names
                    #print(keys)
                    id = int(id[0][0]) - 1  # MATLAB is 1-origin
                    pos = pos[0].tolist()
                    occl = int(occl[0][0])
                    lock = int(lock[0][0])
                    posv = posv[0].tolist()

                    datum = dict(zip(keys, [id, pos, occl, lock, posv]))
                    datum['lbl'] = str(objLbl[datum['id']])
                    datum['str'] = int(objStr[datum['id']])
                    datum['end'] = int(objEnd[datum['id']])
                    datum['hide'] = int(objHide[datum['id']])
                    datum['init'] = int(objInit[datum['id']])
                    data[set_name][video_name][
                        'frames'][frame_id].append(datum)
                    n_obj += 1

        print(dname, anno_fn, n_obj)
        all_obj += n_obj

print('Number of objects:', all_obj)
json.dump(data, open('data/annotations.json', 'w'))

data/annotations/set00 data/annotations/set00/V000.vbb 4519
data/annotations/set00 data/annotations/set00/V001.vbb 7303
data/annotations/set00 data/annotations/set00/V002.vbb 2334
data/annotations/set00 data/annotations/set00/V003.vbb 642
data/annotations/set00 data/annotations/set00/V004.vbb 1879
data/annotations/set00 data/annotations/set00/V005.vbb 97
data/annotations/set00 data/annotations/set00/V006.vbb 7203
data/annotations/set00 data/annotations/set00/V007.vbb 10449
data/annotations/set00 data/annotations/set00/V008.vbb 4978
data/annotations/set00 data/annotations/set00/V009.vbb 6472
data/annotations/set00 data/annotations/set00/V010.vbb 7056
data/annotations/set00 data/annotations/set00/V011.vbb 7132
data/annotations/set00 data/annotations/set00/V012.vbb 3622
data/annotations/set00 data/annotations/set00/V013.vbb 7748
data/annotations/set00 data/annotations/set00/V014.vbb 9031
data/annotations/set01 data/annotations/set01/V000.vbb 4488
data/annotations/set01 data/annotations/se

In [14]:
import cv2 as cv
import struct

def seq_to_images(dname,path):
    def read_header(ifile):
        feed = ifile.read(4)
        norpix = ifile.read(24)
        version = struct.unpack('@i', ifile.read(4))
        length = struct.unpack('@i', ifile.read(4))
        assert(length != 1024)
        descr = ifile.read(512)
        params = [struct.unpack('@i', ifile.read(4))[0] for i in range(9)]
        fps = struct.unpack('@d', ifile.read(8))
        ifile.read(432)
        image_ext = {100:'raw', 102:'jpg', 201:'jpg', 1:'png', 2:'png'}
        return {'w':params[0], 'h':params[1], 'bdepth':params[2],
                'ext':image_ext[params[5]], 'format':params[5],
                'size':params[4], 'true_size':params[8],
                'num_frames':params[6]}

    assert path[-3:] == 'seq', path
    ifile = open(path, 'rb')
    params = read_header(ifile)
    bytes = open(path, 'rb').read()

    #imgs = []
    extra = 8
    s = 1024
    for i in range(params['num_frames']):
        tmp = struct.unpack_from('@I', bytes[s:s+4])[0]
        I = bytes[s+4:s+tmp]
        s += tmp + extra
        if i == 0:
            val = struct.unpack_from('@B', bytes[s:s+1])[0]
            if val != 0:
                s -= 4
            else:
                extra += 8
                s += 8

        tmp_file = '/tmp/img%d.jpg' % i
        open(tmp_file, 'wb+').write(I)
        img = cv.imread(tmp_file)
        #imgs.append(img)
        save_img(dname, fn, i, img)

    return i

def save_img(dname, fn, i, frame):
    #print("I am in")
    cv.imwrite('{}/{}_{}_{}.png'.format(
        out_dir, os.path.basename(dname),
        os.path.basename(fn).split('.')[0], i), frame)

out_dir = 'data/images'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
for dname in sorted(glob.glob('data/set*')):
    for fn in sorted(glob.glob('{}/*.seq'.format(dname))):
#         if fn.split("/")[1] == "set"
        #cap = cv.VideoCapture(fn)
        seq_to_images(dname, fn)
  
        print(fn)


data/set00/V000.seq
data/set00/V001.seq
data/set00/V002.seq
data/set00/V003.seq
data/set00/V004.seq
data/set00/V005.seq
data/set00/V006.seq
data/set00/V007.seq
data/set00/V008.seq
data/set00/V009.seq
data/set00/V010.seq
data/set00/V011.seq
data/set00/V012.seq
data/set00/V013.seq
data/set00/V014.seq
data/set01/V000.seq
data/set01/V001.seq
data/set01/V002.seq
data/set01/V003.seq
data/set01/V004.seq
data/set01/V005.seq
data/set02/V000.seq
data/set02/V001.seq
data/set02/V002.seq
data/set02/V003.seq
data/set02/V004.seq
data/set02/V005.seq
data/set02/V006.seq
data/set02/V007.seq
data/set02/V008.seq
data/set02/V009.seq
data/set02/V010.seq
data/set02/V011.seq
data/set03/V000.seq
data/set03/V001.seq
data/set03/V002.seq
data/set03/V003.seq
data/set03/V004.seq
data/set03/V005.seq
data/set03/V006.seq
data/set03/V007.seq
data/set03/V008.seq
data/set03/V009.seq
data/set03/V010.seq
data/set03/V011.seq
data/set03/V012.seq
data/set04/V000.seq
data/set04/V001.seq
data/set04/V002.seq
data/set04/V003.seq


OSError: [Errno 28] No space left on device